# Attempt 1

* NaNs replaced by zeros
* Using QuantileTransformer on target
* Using random 80/20 train/test split
* XGBoost on RF using parameters from GridSearch

In [6]:
import numpy as np
import pandas as pd
import matplotlib
import math
from sklearn import model_selection, ensemble, metrics, linear_model, preprocessing
from matplotlib import pyplot as plt
%matplotlib inline

In [7]:
data = pd.read_csv('dataset/train.csv')

In [8]:
test = pd.read_csv('dataset/test.csv')

In [9]:
#Test dataset contains objects only of latest 10 years,
#including unmentioned in train data latest year 1016064
unique_years = data['galactic year'].unique()
print (unique_years)
print (unique_years.shape)

print(np.sort(test['galactic year'].unique()))
print(test['galactic year'].unique().shape)

unique_names = data['galaxy'].unique()
unique_names = np.sort(unique_names)
print (unique_names[:5])
print(unique_names.shape)

[ 990025  991020  992016  993012  994009  995006  996004  997002  998001
  999000 1000000 1001000 1002001 1003002 1004004 1005006 1006009 1007012
 1008016 1009020 1010025 1011030 1012036 1013042 1014049 1015056]
(26,)
[1007012 1008016 1009020 1010025 1011030 1012036 1013042 1014049 1015056
 1016064]
(10,)
['Andromeda Galaxy (M31)' 'Andromeda I' 'Andromeda II' 'Andromeda III'
 'Andromeda IX']
(181,)


In [10]:
#Let's map all galactic years to years with increment of 1 year
# and map all galaxies to their unique integer key

#dictionary for replacement of galactic years with normal years
di = {val: ind+1 for ind, val in enumerate(np.append(unique_years,
                                                     1016064))}
#dictionary for replacement of names with integer keys
di_names = {val: ind+1 for ind, val in enumerate(unique_names)}
data.replace({'galactic year':di}, inplace=True)
test.replace({'galactic year':di}, inplace=True)

data.replace({'galaxy':di_names}, inplace=True)
test.replace({'galaxy':di_names}, inplace=True)

In [12]:
from sklearn.model_selection import cross_val_score as CVS
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_squared_error as MSE
from sklearn.compose import TransformedTargetRegressor as TransTargReg
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import QuantileTransformer, quantile_transform

# getting X and y
X = data.iloc[:, 0:-1].values
y = data.iloc[:, -1].values

imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
X = imputer.fit_transform(X)

#creating a target transformer (normal)
qt = QuantileTransformer(n_quantiles=1000, output_distribution='normal' )

#normalize
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)

(X_train, X_test, y_train,
 y_test) = model_selection.train_test_split(X, y,
          test_size = 0.2, shuffle=True, random_state=425)

In [14]:
import xgboost as xgb
XGB = TransTargReg(xgb.XGBRegressor(
    max_depth=5,
    learning_rate=0.1,
    n_estimators=500,
    verbosity=1,
    silent=None,
    objective='reg:squarederror',
    booster='gbtree',
    n_jobs=3,
    nthread=None,
    gamma=0,
    min_child_weight=1,
    max_delta_step=0,
    subsample=0.5,
    colsample_bytree=1,
    colsample_bylevel=1,
    colsample_bynode=0.3,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    base_score=0.5,
    random_state=42,
    seed=None,
    missing=None,
    importance_type='gain'),
                   qt)
XGB.fit(X_train, y_train)

print(MSE(y_train, XGB.predict(X_train))**0.5)
print(MSE(y_test, XGB.predict(X_test))**0.5)

0.004783013588373652
0.019910947597162425
